# A notebook for texting Alpaca API

## Define class for interfacing with Alpaca
This will use WebSockets to:
* Save last quotes in dict

In [1]:
import alpaca_trade_api as tradeapi
import pandas as pd

class AlpacaDataSubscriber:
    __con = None
    
    def __init__(self):
        """
        Get the connection params from vault and create the stream connection
        """
        con_params = {}
        with open(".alpaca-paper.vault") as f:
            for line in f:
               (key, val) = line.split("=")
               con_params[key] = val.rstrip()
                
        self.__con = tradeapi.stream.Stream(con_params['key'], con_params['secret-key'], base_url=con_params['url'], data_feed='iex')
        
    def subscribe_quotes(self, symbol: str):
        """
        Subscribe symbol for quote updates
        :param str symbol: The symbol to subscribe 
        """
        self.__con.subscribe_quotes(self.__on_quote, symbol)
        
    def unsubscribe_quotes(self, symbol: str):
        """
        Unubscribe symbol for quote updates
        :param str symbol: The symbol to unsubscribe 
        """
        self.__con.unsubscribe_quotes(symbol)
        
    def subscribe_trades(self, symbol: str):
        """
        Subscribe symbol for trade updates
        :param str symbol: The symbol to subscribe 
        """
        self.__con.subscribe_trades(self.__on_trade, symbol)
        
    def unsubscribe_trades(self, symbol: str):
        """
        Unubscribe symbol for trade updates
        :param str symbol: The symbol to unsubscribe 
        """
        self.__con.unsubscribe_trades(symbol)
        
    def subscribe_candles(self, symbol: str, period: str):
        """
        Subscribe symbol for candle updates
        :param str symbol: The symbol to subscribe
        :param str period: m=minute; d=daily
        """
        if period == 'm':
            self.__con.subscribe_bars(self.__on_bar_1m, symbol)
        elif period == 'd':
            self.__con.subscribe_daily_bars(self.__on_bar_1d, symbol)
        
    def unsubscribe_candles(self, symbol: str, period: str):
        """
        Unubscribe symbol for candle updates
        :param str symbol: The symbol to unsubscribe
        :param str period: b=minute; d=daily
        """
        if period == 'm':
            self.__con.unsubscribe_bars(symbol)
        elif period == 'd':
            self.__con.unsubscribe_daily_bars(symbol)
                       
    def subscribe_status(self, symbol: str):
        """
        Subscribe symbol for status updates
        :param str symbol: The symbol to subscribe
        :param str period: m=minute; d=daily
        """
        self.__con.subscribe_statuses(self.__on_status, symbol)
        
    def unsubscribe_status(self, symbol: str):
        """
        Unubscribe symbol for status updates
        :param str symbol: The symbol to unsubscribe
        """
        self.__con.unsubscribe_statuses(symbol)
        
    def subscribe_news(self, symbol: str):
        """
        Subscribe symbol for news updates
        :param str symbol: The symbol to subscribe
        :param str period: m=minute; d=daily
        """
        self.__con.subscribe_news(self.__on_news, symbol)
        
    def unsubscribe_news(self, symbol: str):
        """
        Unubscribe symbol for news updates
        :param str symbol: The symbol to unsubscribe
        """
        self.__con.unsubscribe_news(symbol)
        
    def start(self):
        """
        Start listening for subscriptions
        """
        self.__con.run()
        
    def stop(self):
        """
        Stop listening for subscriptions
        """
        self.__con.stop()
        
    async def __on_quote(self, quote):
        """
        Called when we recieve a new quote. We will:
        1) Log to console
        2) ...
        """     
        
        # Get the quote as a dict
        quote = quote._raw
        
        # Convert the timestamp to UTC in pandas format 
        quote['timestamp'] = pd.Timestamp(quote['timestamp'], tz='UTC')
        
        # Log
        print(f"__on_quote quote: {quote}")
    
    async def __on_trade(self, trade):
        """
        Called when we recieve a new trade. We will:
        1) Log to console
        2) ...
        """
        
        # Get the trade as a dict
        trade = trade._raw
        
        # Convert the timestamp to UTC in pandas format 
        trade['timestamp'] = pd.Timestamp(trade['timestamp'], tz='UTC')
        
        # Log
        print(f"__on_trade trade: {trade}")
        
    async def __on_bar_1m(self, bar):
        """
        Called when we recieve a new 1m bar. Callls the __on_bar method with a period of 'm'
        """ 
        self.__on_bar(bar, 'm')
        
    async def __on_bar_1d(self, bar):
        """
        Called when we recieve a new 1d bar. Callls the __on_bar method with a period of 'd'
        """ 
        self.__on_bar(bar, 'd')
        
    def __on_bar(self, bar, period):
        """
        Called when we recieve a new bar. We will:
        1) Log to console
        2) ...
        """     
        
        # Get the bar as a dict
        bar = bar._raw
        
        # Convert the timestamp to UTC in pandas format 
        bar['timestamp'] = pd.Timestamp(bar['timestamp'], tz='UTC')
        
        # Log
        print(f"__on_bar bar: {bar} period {period}")
        
    async def __on_status(self, status):
        """
        Called when we recieve a new status update. We will:
        1) Log to console
        2) ...
        """     
        
        # Get the status as a dict
        status = status._raw
        
        # Convert the timestamp to UTC in pandas format 
        status['timestamp'] = pd.Timestamp(status['timestamp'], tz='UTC')
        
        # Log
        print(f"__on_status status: {status}")
        
    async def __on_news(self, news):
        """
        Called when we recieve news. We will:
        1) Log to console
        2) ...
        """     
        
        # Get the news as a dict
        news = news._raw
        
        # Convert the timestamp to UTC in pandas format 
        news['timestamp'] = pd.Timestamp(news['timestamp'], tz='UTC')
        
        # Log
        print(f"__on_news news: {news}")
        

## Test it

In [ ]:
import nest_asyncio

# Only needed in Jupyter
nest_asyncio.apply()

al = Alpaca()
al.subscribe_quotes('IBM')
al.unsubscribe_quotes('IBM')
#al.subscribe_quotes('TSLA')
#al.subscribe_candles('TSLA', 'm')
#al.subscribe_candles('TSLA', 'd')
#al.subscribe_trades('TSLA')
al.subscribe_status('TSLA')
al.subscribe_news('TSLA')

al.start()